In [ ]:
from aux_functions import *
from sphere_vector_kernels import *
from sphere_vector_gp import *
from blender_file_generation import *

import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.gaussian_process.kernels import ConstantKernel

In [ ]:
def plot_samples(lon, lat, samples, nrows, ncols, suptitle=""):
    assert samples.shape[2] <= nrows * ncols, "Not enough subplots to display all samples!"
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5 * nrows, 5 * ncols))

    for k in range(samples.shape[2]):
        i, j = k // ncols, k % ncols
        
        sample = samples[:, :, k]

        axs[i, j].quiver(lon, lat, sample[:, 1], sample[:, 0], angles="uv")
        axs[i, j].set_xlabel("lon")
        axs[i, j].set_ylabel("lat")
        axs[i, j].set_title(f"Sample {k + 1}")

    fig.suptitle(suptitle)
    fig.tight_layout()
    return fig

## Read data

Blender input data points.

In [ ]:
blender_folder = "blender-data"
out_folder = os.path.join("blender-data", "outputs")

mean_inputs = pd.read_csv(
    os.path.join(blender_folder, "mean_inputs.csv"),
    names=["x", "y", "z"]
).to_numpy()

In [ ]:
mean_inputs_sph = car_to_sph(mean_inputs)

## GPs

GPs with variance 1 for the differnet kernels.

In [ ]:
kappa = 10

k_proj = ProjectedSphereKernel(kappa=kappa)
k_hodge = HodgeSphereKernel(kappa=kappa)

In [ ]:
# find normalizations
norm_proj = k_proj(np.array([[0., 0.]], dtype="float64"))[0, 0]
norm_hodge = k_hodge(np.array([[0., 0.]], dtype="float64"))[0, 0]

In [ ]:
k_proj = ConstantKernel(1 / norm_proj) * k_proj
k_hodge = ConstantKernel(1 / norm_hodge) * k_hodge

In [ ]:
gp_proj = SphereVectorGP(kernel=k_proj, n_restarts_optimizer=0)
gp_hodge = SphereVectorGP(kernel=k_hodge, n_restarts_optimizer=0)

## Generate samples

In [ ]:
n_samples = 6

In [ ]:
samples_hodge = gp_hodge.sample_y(mean_inputs_sph, n_samples=n_samples)

In [ ]:
samples_proj = gp_proj.sample_y(mean_inputs_sph, n_samples=n_samples)

In [ ]:
fig = plot_samples(mean_inputs_sph[:, 1], mean_inputs_sph[:, 0], samples_hodge, (n_samples + 1)//2, 2, suptitle=f"Samples from Hodge kernel ({kappa=})")

In [ ]:
fig = plot_samples(mean_inputs_sph[:, 1], mean_inputs_sph[:, 0], samples_proj, (n_samples + 1)//2, 2, suptitle=f"Samples from projected kernel ({kappa=})")

In [ ]:
out_folder = os.path.join("blender-data", "outputs")
for k in range(n_samples):
    sp, sh = samples_proj[:, :, k], samples_hodge[:, :, k]
    _, sp = v_sph_to_car(mean_inputs_sph, sp)
    _, sh = v_sph_to_car(mean_inputs_sph, sh)
    
    np.savetxt(os.path.join(out_folder, f"proj_prior_sample_{kappa=}_{k}__mean.csv"), np.hstack([mean_inputs, sp]), delimiter=",")
    np.savetxt(os.path.join(out_folder, f"hodge_prior_sample_{kappa=}_{k}__mean.csv"), np.hstack([mean_inputs, sh]), delimiter=",")

### dif-free and curl-free

In [ ]:
# div free
n_samples = 6
kappa = 10
k = HodgeMaternDivFreeSphereKernel(kappa=kappa)
norm = k(np.array([[0., 0.]], dtype="float64"))[0, 0]
k = ConstantKernel(1 / norm) * k
gp = SphereVectorGP(kernel=k, n_restarts_optimizer=0)
print("Sampling...")
samples = gp.sample_y(mean_inputs_sph, n_samples=n_samples)
print("Done!")
fig = plot_samples(
    mean_inputs_sph[:, 1], mean_inputs_sph[:, 0],
    samples, (n_samples + 1)//2, 2,
    suptitle=f"Samples div-free ({kappa=})"
)

out_folder = os.path.join("blender-data", "outputs")
for k in range(samples.shape[-1]):
    s = samples[:, :, k]
    _, s = v_sph_to_car(mean_inputs_sph, s)
    
    np.savetxt(os.path.join(out_folder, f"sample_div_free_{k}__mean.csv"), np.hstack([mean_inputs, s]), delimiter=",")

In [ ]:
# curl free
n_samples = 6
kappa = 10
k = HodgeMaternCurlFreeSphereKernel(kappa=kappa)
norm = k(np.array([[0., 0.]], dtype="float64"))[0, 0]
k = ConstantKernel(1 / norm) * k
gp = SphereVectorGP(kernel=k, n_restarts_optimizer=0)
print("Sampling...")
samples = gp.sample_y(mean_inputs_sph, n_samples=n_samples)
print("Done!")
fig = plot_samples(
    mean_inputs_sph[:, 1], mean_inputs_sph[:, 0],
    samples, (n_samples + 1)//2, 2,
    suptitle=f"Samples curl-free ({kappa=})"
)

out_folder = os.path.join("blender-data", "outputs")
for k in range(samples.shape[-1]):
    s = samples[:, :, k]
    _, s = v_sph_to_car(mean_inputs_sph, s)
    
    np.savetxt(os.path.join(out_folder, f"sample_curl_free_{k}__mean.csv"), np.hstack([mean_inputs, s]), delimiter=",")